# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df= pd.read_csv("city_weather.csv")
del city_df["Unnamed: 0"]
city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,ust-kamchatsk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sorong,94.0,ID,1.650823e+09,83.0,-0.8833,131.25,81.23,9.69
2,ribeira grande,40.0,PT,1.650823e+09,67.0,38.5167,-28.70,63.00,17.27
3,ushuaia,99.0,AR,1.650823e+09,66.0,-54.8000,-68.30,43.41,3.38
4,samusu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Handle NaN values
city_df = city_df.dropna()
humidity = city_df["Humidity"].astype(float)
# Locations
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]


In [4]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(city_df["Humidity"]),
                                 point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# # Narrow down the DataFrame to find your ideal weather condition.
narrowed_city_df = city_df

# # Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# # A max temperature lower than 80 degrees but higher than 70.
narrowed_city_df = narrowed_city_df.loc[(narrowed_city_df["Max Temp"] < 80) & (narrowed_city_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
narrowed_city_df= narrowed_city_df.loc[narrowed_city_df["Windspeed"] < 10]

# # Zero cloudiness.
narrowed_city_df = narrowed_city_df.loc[narrowed_city_df["Cloudiness"] == 0]

# # Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
9,lashio,0.0,MM,1.650823e+09,52.0,22.9333,97.7500,70.47,0.81
20,laguna,0.0,US,1.650823e+09,48.0,38.4210,-121.4238,72.97,3.44
216,broome,0.0,US,1.650823e+09,45.0,42.2506,-75.8330,77.70,7.00
307,tura,0.0,IN,1.650824e+09,73.0,25.5198,90.2201,74.75,6.20
354,guerrero negro,0.0,MX,1.650824e+09,40.0,27.9769,-114.0611,73.06,7.14
439,inhambane,0.0,MZ,1.650824e+09,82.0,-23.8650,35.3833,74.19,6.15
481,lompoc,0.0,US,1.650824e+09,57.0,34.6391,-120.4579,79.70,4.00
515,ixtapa,0.0,MX,1.650824e+09,65.0,20.7000,-105.2000,79.83,9.22
530,erzin,0.0,TR,1.650824e+09,44.0,36.9559,36.2000,72.59,2.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,lashio,MM,22.9333,97.7500,
20,laguna,US,38.4210,-121.4238,
216,broome,US,42.2506,-75.8330,
307,tura,IN,25.5198,90.2201,
354,guerrero negro,MX,27.9769,-114.0611,
439,inhambane,MZ,-23.8650,35.3833,
481,lompoc,US,34.6391,-120.4579,
515,ixtapa,MX,20.7000,-105.2000,
530,erzin,TR,36.9559,36.2000,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
     # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    # Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 9: lashio.
Closest hotel in lashio is Two Elephants Hotel.
------------
Retrieving Results for Index 20: laguna.
Closest hotel in laguna is Hampton Inn & Suites Sacramento-Elk Grove Laguna I-5.
------------
Retrieving Results for Index 216: broome.
Missing field/result... skipping.
------------
Retrieving Results for Index 307: tura.
Closest hotel in tura is Hotel Polo Orchid.
------------
Retrieving Results for Index 354: guerrero negro.
Closest hotel in guerrero negro is The Halfway Inn.
------------
Retrieving Results for Index 439: inhambane.
Closest hotel in inhambane is Hotel Casa Do Capitão.
------------
Retrieving Results for Index 481: lompoc.
Closest hotel in lompoc is Hilton Garden Inn Lompoc.
------------
Retrieving Results for Index 515: ixtapa.
Closest hotel in ixtapa is Flamingo Vallarta Hotel & Marina.
------------
Retrieving Results for Index 530: erzin.
Closest hotel in erzin is Hattusa Vacation Thermal Club Erzin.
------------
-------End 

In [9]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,lashio,MM,22.9333,97.7500,Two Elephants Hotel
20,laguna,US,38.4210,-121.4238,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
216,broome,US,42.2506,-75.8330,
307,tura,IN,25.5198,90.2201,Hotel Polo Orchid
354,guerrero negro,MX,27.9769,-114.0611,The Halfway Inn
439,inhambane,MZ,-23.8650,35.3833,Hotel Casa Do Capitão
481,lompoc,US,34.6391,-120.4579,Hilton Garden Inn Lompoc
515,ixtapa,MX,20.7000,-105.2000,Flamingo Vallarta Hotel & Marina
530,erzin,TR,36.9559,36.2000,Hattusa Vacation Thermal Club Erzin


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))